In [11]:
#@title Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

# 1. Authorizing google colab
from google.colab import auth
auth.authenticate_user()

# 2. credentials for google sheets
import gspread
from google.auth import default
creds, _ = default()

# 3. authotizing the connection
gc = gspread.authorize(creds)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#@title import
import random

import numpy as np
import pandas as pd

from time import sleep
from datetime import datetime

In [13]:
#@title parameters
archive_googlesheet_ID = '1WyDXV6AXQGRs0PPzMrzQXm7jdKMCH8bVGBvha97zf1k'
result_googlesheet_ID = '1UZPpqPW_zMDl1-P3JSCb5iJff2AFJYW_6oH8YSzCbgc'
results_sheet_1 = 'PSO_1'
#default_save_path = "/content/drive/MyDrive/"
backup_path = "/content/drive/MyDrive/PNaF/Diamentowy_Grant_2017-2021/Swimming_caterpillars_evolution/wyniki/backup/"

parameters_names = ['frequency [Hz]', 'length [mm]', 'dye [%mol]', 'thickness [µm]', 'alignment period [mm]',
             'laser power [W]', 'right side [down 0, up 1]', 'waveplate angle [deg]']

PSO_velocity = ['parameter {} velocity'.format(i+1) for i in range(len(parameters_names))]

fitness_names = ['fitness function \n(max average)', 'best fitness function']

individuals_no = 8
iteration_max = 5

frequency_list = [float(i)/10. for i in range(1,51)]
length_list = [6,12,18]
dye_list = [0.2,1]
thickness_list = [50,90]
period_list = [1,2,3]
laserPower_list = [float(i)/2. for i in range(1,9)]
waveplate_list = [7.5*int(i)+30 for i in range(0,12)]
# right side of the film 0 down, 1 up
tail_list = [0,1]
parameters_list = [frequency_list, length_list, dye_list, thickness_list,
                   period_list, laserPower_list, tail_list, waveplate_list]

c0_global = 0. # 0.4
c1_global = 0.2 # 2.5
c2_global = 2.2 # 0.5


# #do testów
# # 14 400 possibilities
# frequency_list = [float(i)/10. for i in range(1,51)]
# length_list = [6,12,18]
# dye_list = [0.2,1]
# thickness_list = [50,90]
# period_list = [1,2,3]
# laserPower_list = [float(i)/2. for i in range(1,9)]
# waveplate_list = [5*int(i)+30 for i in range(0,18)]
# # right side of the film 0 down, 1 up
# tail_list = [0,1]

# #test
# frequency_list = [float(i) for i in range(0,120)]
# laserPower_list = [float(i) for i in range(0,120)]


# parameters_list = [frequency_list, length_list, dye_list, thickness_list,
#                    period_list, laserPower_list, tail_list, waveplate_list]


In [14]:
#@title definitions

#działa, opisana
def get_single_sheet_by_name_dataframe(workbook, sheet_name):
    worksheet = workbook.worksheet(sheet_name)
    rows = worksheet.get_all_values()
    df = pd.DataFrame(rows)
    df.index = df.copy()[0]
    df = df[df.columns[1:]]
    return df.copy()

#działa, opisana
def get_single_sheet_by_number_dataframe(workbook, sheet_no):
    worksheet = workbook.get_worksheet(sheet_no)
    rows = worksheet.get_all_values()
    df = pd.DataFrame(rows)
    df.index = df.copy()[0]
    df = df[df.columns[1:]]
    return df.copy()

#działa, opisana
def get_googlesheet_by_name_dataframe(googlesheet_ID, sheet_name = 'results'):
    workbook = gc.open_by_key(googlesheet_ID)
    worksheet = workbook.worksheet(sheet_name)
    rows = worksheet.get_all_values()
    df = pd.DataFrame(rows)
    df.index = df.copy()[0]
    df = df[df.columns[1:]]
    return df.copy()

#działa, opisana
def merge_dataframes_list(dataframes_list):
    dataframes_no = len(dataframes_list)
    merged_dataframe = dataframes_list[0]
    index = list(merged_dataframe.index.values)
    merged_dataframe = merged_dataframe.reset_index(drop=True)
    for i in range(dataframes_no-1):
        #merged_dataframe = pd.concat([merged_dataframe, dataframes_list[i+1]], axis=1)
        merged_dataframe = pd.concat([merged_dataframe, dataframes_list[i+1].reset_index(drop=True)], axis=1, ignore_index=True)
    for i in range(len(index)):
        merged_dataframe.rename(index={list(merged_dataframe.index.values)[i]:index[i]},inplace=True)
    return merged_dataframe.copy()

#działa, opisana
def write_dataframe_to_googlesheet(dataframe, googlesheet_ID, sheet_name, row_names = True, column_names = False):
    data = dataframe.copy().fillna('')
    #dataframe = dataframe.fillna(0, inplace=True)
    if (row_names):
        data.insert(0, '0', data.index, True)
    if (column_names):
        column_names_list = data.columns.to_list()
        data = data.to_numpy().tolist()
        all_data = [column_names_list] + data
    else:
        all_data = data.to_numpy().tolist()
    workbook = gc.open_by_key(googlesheet_ID)
    worksheet = workbook.worksheet(sheet_name)
    MAX_ROWS = 10
    for i in range(0, len(all_data), MAX_ROWS):
        chunk = all_data[i:i+MAX_ROWS]
        range_start = i + 1
        worksheet.update(range_name=f"A{range_start}", values=chunk)
    # worksheet.update(None,all_data)
    return

#działa, opisana
def get_all_sheets_from_googlesheet_dataframeList(googlesheet_ID):
    workbook = gc.open_by_key(googlesheet_ID)
    df = []
    sheets_no = len(workbook.worksheets())
    for i in range(sheets_no):
        df.append(get_single_sheet_by_number_dataframe(workbook, i).copy())
    return df

#działa, opisana
def get_txt_by_path_list(path):
    with open(path, "r") as f:
        previous_results = f.readlines()
    return previous_results

#działa, opisana
def find_index(index_name):
    for i, name in enumerate(parameters_names):
        if index_name in name:
            return i
    print("No index found")
    return

#działa, opisana
def waveplate_periodic_boundary_conditions(parameters):
    i = find_index("waveplate")
    waveplate = parameters[i]
    waveplate_list = parameters_list[i]
    delta = abs(waveplate_list[1] - waveplate_list[0])
    waveplate_floor = np.floor(waveplate/delta)

    if 2*(waveplate - waveplate_floor*delta) >= delta:
        waveplate = (waveplate_floor+1) * delta
    else:
        waveplate = waveplate_floor * delta
    if abs(waveplate - min(waveplate_list)) < delta/3.:
        waveplate = min(waveplate_list)
    elif abs(waveplate - max(waveplate_list)) < delta/3.:
        waveplate = max(waveplate_list)

    while waveplate > max(waveplate_list):
        waveplate -= (max(waveplate_list) - min(waveplate_list))
    while waveplate < min(waveplate_list):
        waveplate += (max(waveplate_list) - min(waveplate_list))

    param = np.copy(parameters)
    param[i] = waveplate
    return param

class caterpillar:
    #names = ["frequency", "length", "dye", "thickness", "period", "laserPower", "tail", "waveplate"]
    #global variables
    parameters = parameters_best = velocities = [0 for i in range (0, len(parameters_names))]
    fitness = fitness_best = 0

    def __init__(self, param = [''], f = -1, v = [''], p_best = [''], f_best = -1):
        self.fitness = f

        if '' in param:
            self.parameters = self.parameters_best = np.array([random.choice(parameters_list[i]) for i in range(0, len(parameters_names))])
            self.velocities = [0. for i in range(0, len(parameters_names))]
            #self.parameters = np.copy(self.parameters)
            #self.velocities = np.copy(self.velocities)
            self.parameters_best = np.copy(self.parameters_best)

        else:
            if f == '':
                f_new = -1
            else:
                f_new = f

            parameters = np.array(param).astype(float)
            self.parameters = np.copy(np.array(param).astype(float))
            if '' in v:
                self.velocities = np.copy(np.array([v]))
            else:
                self.velocities = np.copy(np.array(v).astype(float))
            f_float = float(f_new)
            f_best_float = float(f_best)
            if f_float > f_best_float or f_best_float == -1.:
                self.parameters_best = np.copy(parameters)
                self.fitness_best = f_float
            else:
                self.parameters_best = np.copy(np.array(p_best).astype(float))
                self.fitness_best = f_best_float

    def change_parameters(self, param, f, v, p_best=[''], f_best=-1):
        self.parameters = np.copy(param)
        self.fitness = f
        self.velocities = np.copy(v)

        if type(f_best) == str or f > f_best or f_best == '':
            self.parameters_best = np.copy(param)
            self.fitness_best = f
        else:
            if '' in p_best:
                self.parameters_best = np.copy(param)
                self.fitness_best = f
            else:
                self.parameters_best = np.copy(p_best)
                self.fitness_best = f_best

    def calc_velocities(self, PSO_params, param_best_global, f_global_best, iteration_no, iteration_max):
        # iteration_no_int = int(iteration_no-1)
        # iteration_max_int = int(iteration_max-1)
        w = PSO_params[0]
        c1 = PSO_params[1]
        c2 = PSO_params[2]
        if '' in self.velocities:
            v = np.array([0 for i in range(len(self.parameters))])
        else:
            v = np.copy(self.velocities)
        if (self.fitness == f_global_best or np.all(self.parameters == param_best_global)) and (iteration_no == 1 or w == 0):
            # v_new = np.array([random.uniform(-1., 1.)*(max(param_list)-min(param_list)) for param_list in parameters_list])
            r = random.uniform(-1., 1.)
            print("r: {}".format(r))
            v_new = np.array([r*(max(param_list)-min(param_list)) for param_list in parameters_list])
        else:
            r1 = random.uniform(0, 1)
            r2 = random.uniform(0, 1)
            v_new = w*v + c1*r1*(self.parameters_best-self.parameters) + c2*r2*(param_best_global-self.parameters)
            waveplate_i = find_index("waveplate")
            # numpy is faster
            for i,vel in enumerate(v_new):
                if i != waveplate_i and np.abs(vel) > max(parameters_list[i])-min(parameters_list[i]):
                    v_new[i] = np.sign(vel)*(max(parameters_list[i])-min(parameters_list[i]))
        self.velocities = np.copy(v_new)
        return

    def step(self, PSO_params, param_best_global, f_global_best, iteration_no, iteration_max):
        self.calc_velocities(PSO_params, param_best_global, f_global_best, iteration_no, iteration_max)
        param = np.copy(self.parameters)
        v = np.copy(self.velocities)
        # p_best = np.copy(self.parameters_best)
        param_new = param + v
        param_new = waveplate_periodic_boundary_conditions(param_new)
        param_new = [min(parameters_list[i], key=lambda x:abs(x-param_new[i])) for i in range (len(parameters_list))]
        self.change_parameters(param_new, -1, v, self.parameters_best, self.fitness_best)

    def print_current_parameters(self):
        parameters_string = ''
        for param in self.parameters:
            parameters_string = parameters_string + str(param) + ' '
        print(parameters_string)
        return

    def print_all_parameters(self):
        parameters_string = 'parameters: '
        for param in self.parameters:
            parameters_string += str(param) + ' '
        parameters_string += '\n'
        parameters_string += 'fitness: '
        parameters_string += str(self.fitness) + '\n'
        parameters_string += 'velocities: '
        for v in self.velocities:
            parameters_string += str(v) + ' '
        parameters_string += '\n'
        parameters_string += 'best parameters: '
        for param in self.parameters_best:
            parameters_string += str(param) + ' '
        parameters_string += '\n'
        parameters_string += 'best fitness: '
        parameters_string += str(self.fitness_best) + ' '
        parameters_string += '\n'
        print(parameters_string)
        return

    def get_copy(self):
        cater = caterpillar(self.parameters, self.fitness, self.velocities, self.parameters_best, self.fitness_best)
        return cater

#działa, opisana
def dataframe_to_caterpillars_list(dataframe):
    df = dataframe.copy().replace(',','.', regex=True)
    caterpillars_list = []
    for column_index in dataframe[df.columns]:
        column = df[column_index]
        parameters = [column[name][0] for name in parameters_names]
        f = column[fitness_names[0]]
        if f != '':
            f = float(f)
        else:
            f = -1
        PSO_v = [column[name] for name in PSO_velocity]
        parameters_best = [column[name][1] for name in parameters_names]
        f_best = column[fitness_names[1]]
        if f_best != '':
            f_best = float(f_best)
        else:
            f_best = -1
        cater = caterpillar(parameters, f, PSO_v, parameters_best, f_best)
        caterpillars_list.append(cater.get_copy())
    return caterpillars_list

#działa, opisana
def get_best_caterpillar(caterpillars_list):
    caterpillar_best = caterpillars_list[0].get_copy()
    for caterpillar in caterpillars_list:
        if caterpillar.fitness_best > caterpillar_best.fitness_best:
            caterpillar_best = caterpillar.get_copy()
    return caterpillar_best

#działa, opisana
def get_parameters_indexes_dictionary(dataframe):
    dataframe_new = dataframe.copy()
    index_list = list(dataframe_new.index)
    velocities_index = index_list.index('parameter 1 velocity')
    parameters_index, best_parameters_index = [i for i, x in enumerate(index_list) if x == parameters_names[0]]
    best_fitness_function_index = [i for i, x in enumerate(index_list) if x == 'best fitness function'][0]
    parameters_indexes_dictionary = {'velocities_index': velocities_index, 'parameters_index': parameters_index, 'best_parameters_index': best_parameters_index, 'best_fitness_function_index': best_fitness_function_index}
    return parameters_indexes_dictionary

#działa, opisana
def update_best_parameters(last_iteration_dataframe, velocities_list, best_parameters_list, best_fitness_list):
    dct = get_parameters_indexes_dictionary(last_iteration_dataframe)
    last_iteration_dataframe_new = last_iteration_dataframe.copy()
    last_iteration_dataframe_new.iloc[dct['velocities_index']:dct['velocities_index']+len(parameters_names),-individuals_no:] = velocities_list
    last_iteration_dataframe_new.iloc[dct['best_parameters_index']:dct['best_parameters_index']+len(parameters_names),-individuals_no:] = best_parameters_list
    last_iteration_dataframe_new.iloc[dct['best_parameters_index']+len(parameters_names),-individuals_no:].fillna(0, inplace=True)
    last_iteration_dataframe_new.iloc[dct['best_parameters_index']+len(parameters_names),-individuals_no:] = last_iteration_dataframe_new.iloc[dct['best_parameters_index']+len(parameters_names)-1,-individuals_no:].astype(float)-30.
    last_iteration_dataframe_new.iloc[dct['best_fitness_function_index'],-individuals_no:] = best_fitness_list
    return last_iteration_dataframe_new

#działa, opisana
def add_empty_column_to_dataframe(dataframe):
    dataframe_copy = dataframe.copy()
    columns_no = len(dataframe_copy.columns)
    empty_column = ['' for i in range(len(dataframe_copy))]
    dataframe_copy[str(columns_no)] = empty_column
    return dataframe_copy

#działa, nieopisana
def check_for_duplicates(cater, caterpillars_list):
    duplicate = False
    cater_genes = [round(float(i),1) for i in cater.parameters]
    for c in caterpillars_list:
        c_genes = [round(float(i),1) for i in c.parameters]
        if np.array_equiv(cater_genes, c_genes):
            duplicate = True
            break
    return duplicate

#działa, nieopisana
def single_mutation(cater, mutation_nr):
    genes_list = cater.parameters.copy()
    cater_new = cater.get_copy()
    # draw parameters to mutate
    parameters_indexes = random.sample(range(len(genes_list)), mutation_nr)
    parameters_list_copy = [np.copy(parameters_list[i]) for i in parameters_indexes]
    for i in range(mutation_nr):
        j = parameters_indexes[i]
        parameters_list_copy[i] = np.delete(parameters_list_copy[i], np.where(parameters_list_copy[i] == genes_list[j]))
        genes_list[j] = random.choice(parameters_list_copy[i])
    cater_new.change_parameters(genes_list, cater.fitness, cater.velocities, cater.parameters_best ,cater.fitness_best)
    return cater_new

#działa, nieopisana
def test_duplicates_PSO(new_caterpillar_list):
    checked_duplicates_no = 0
    for c in new_caterpillar_list:
        for cater in new_caterpillar_list:
            if all(c.parameters == cater.parameters):
                checked_duplicates_no += 1
    if checked_duplicates_no != len(new_caterpillar_list):
        for c in new_caterpillar_list:
            c.print_current_parameters()
            print(c.fitness)
        print(checked_duplicates_no)
        print("test exit")
        print("\n")
        exit(checked_duplicates_no)
    return

#działa, nieopisana, nieużywana
def change_duplicates(new_caterpillars_list, old_caterpillars_list):
    checked_caterpillars_list = old_caterpillars_list.copy()
    new_caterpillars_list_without_duplicates = []
    for i, cater in enumerate(new_caterpillars_list):
        c = cater.get_copy()
        j = 0
        while check_for_duplicates(c, checked_caterpillars_list) and j < 5:
            if j == 0:
                print('change duplicates, duplicate detected: {}'.format(c.parameters))
            c.step()
            j += 1
            print('change duplicates, PSO step no: {}'.format(j))
        while check_for_duplicates(c, checked_caterpillars_list):
            print('change duplicates, mutation')
            c = single_mutation(c, 1)
            print(c.print_current_parameters())
        checked_caterpillars_list.append(c.get_copy())
        new_caterpillars_list_without_duplicates.append(c.get_copy())

    test_duplicates_PSO(checked_caterpillars_list)
    return new_caterpillars_list_without_duplicates

#działa, opisana
def update_caterpillar_list(PSO_params, caterpillars_list, iteration_no, iteration_max):
    caterpillar_list_copy = [cater.get_copy() for cater in caterpillars_list]
    caterpillar_best = get_best_caterpillar(caterpillar_list_copy)
    velocities_list = []
    best_parameters_list = []
    best_fitness_list = []
    for cater in caterpillar_list_copy:
        cater.calc_velocities(PSO_params, caterpillar_best.parameters_best, caterpillar_best.fitness_best, iteration_no, iteration_max)
        velocities_list.append(cater.velocities)
        best_parameters_list.append(cater.parameters_best)
        best_fitness_list.append(cater.fitness_best)
    velocities_list = np.array(velocities_list).T
    best_parameters_list = np.array(best_parameters_list).T
    iteration_parameters = {'velocities_list': velocities_list, 'best_parameters_list': best_parameters_list, 'best_fitness_list': best_fitness_list}
    return caterpillar_list_copy, iteration_parameters

#nie wiem czy działa, źle opisana, nieużywana?
def get_new_iteration_caterpillar(caterpillars_list, old_caterpillars_list):
    caterpillars_list_new = [cater.get_copy() for cater in caterpillars_list]
    # caterpillar_best = get_best_caterpillar(caterpillars_list_new)
    for cater in caterpillars_list_new:
        cater.step()
    caterpillars_list_new = change_duplicates(caterpillars_list_new, old_caterpillars_list)
    return caterpillars_list_new

def get_new_iteration(PSO_params, individuals_no, caterpillars_list_all, iteration_no, iteration_max):
    caterpillars_list_new = [cater.get_copy() for cater in caterpillars_list_all[-individuals_no:]]
    caterpillar_best = get_best_caterpillar(caterpillars_list_new)
    caterpillar_best.print_all_parameters()
    for i, cater in enumerate(caterpillars_list_new):
        j = 0
        c = cater.get_copy()
        c.step(PSO_params, caterpillar_best.parameters_best, caterpillar_best.fitness_best, iteration_no, iteration_max)
        while check_for_duplicates(c, caterpillars_list_all) or check_for_duplicates(c, caterpillars_list_new[:i]):
            if j < 5:
                c.step(PSO_params, caterpillar_best.parameters_best, caterpillar_best.fitness_best, iteration_no, iteration_max)
                j += 1
                print('caterpillar {} change duplicates, PSO step no: {}'.format(i+1,j))
            else:
                c = single_mutation(c, 1)
                print('caterpillar {} change duplicates, mutation'.format(i+1))
        caterpillars_list_new[i] = c.get_copy()
    return caterpillars_list_new

# dotąd zrobiłem

#działa, opisana
def caterpillars_list_to_dataframe(caterpillars_list, indexes_dictionary):
    caterpillars_dataframe = pd.DataFrame()
    indexes_dictionary_sorted = sorted(indexes_dictionary.items(), key=lambda item: item[1])
    cater_index = 0
    for cater in caterpillars_list:
        cater_index += 1
        new_column = []
        for index in indexes_dictionary_sorted:
            while len(new_column) < index[1]:
                new_column.append('')
            if index[0] == 'parameters_index':
                for element in cater.parameters:
                    new_column.append(element)
                new_column.append(float(element)-30.)
            elif index[0] == 'velocities_index':
                for element in cater.velocities:
                    new_column.append(element)
            elif index[0] == 'best_parameters_index':
                for element in cater.parameters_best:
                    new_column.append(element)
                new_column.append(float(element)-30.)
            elif index[0] == 'best_fitness_function_index':
                new_column.append(cater.fitness_best)
        caterpillars_dataframe[cater_index] = new_column
    return caterpillars_dataframe

#działa, opisana
def append_caterpillars_list_to_dataframe(dataframe, caterpillars_list):
    dataframe_copy = dataframe.copy()
    indexes_dictionary = get_parameters_indexes_dictionary(dataframe)
    dataframe_with_caterpillars = caterpillars_list_to_dataframe(caterpillars_list, indexes_dictionary)
    dataframe_with_caterpillars.index = dataframe_copy.index

    last_column_index = dataframe_copy.columns[-1]
    dataframe_with_caterpillars.columns = [int(col_name)+int(last_column_index) for col_name in dataframe_with_caterpillars.columns]

    dataframe_new = merge_dataframes_list([dataframe_copy, dataframe_with_caterpillars])
    index_list = list(dataframe_new.index.values)
    robot_no_index = [i for i, x in enumerate(index_list) if x == 'robot no'][0]
    dataframe_new.iloc[robot_no_index, -individuals_no:] = np.array([i+1 for i in range(individuals_no)]).astype(object)

    rows_no, cols_no = dataframe_copy.shape
    last_index = cols_no
    while dataframe_new.iloc[0,last_index] == '':
        last_index -= 1
    dataframe_new.iloc[0,cols_no] = int(dataframe_new.iloc[0,last_index]) + 1

    return dataframe_new

#działa, opisana
def remove_dataframe_duplicates(dataframe, make_index = False):
    data = dataframe.copy()
    if (make_index):
        data.index = data[0]
        data = data.drop(data.columns[0], axis=1)
    data = data.replace(',','.', regex=True)
    # data = data.astype('float')
    data = data.T.drop_duplicates().T
    columns_len = len(data.columns)
    new_column_name = list(range(1,columns_len+1))
    data.columns = new_column_name
    return data

#działa, opisana
def overwrite_googlesheet_by_dataframe(dataframe, googlesheet_ID, sheet_name, row_names = True, column_names = False):
    data = dataframe.copy().fillna('')
    data = data.astype('str')
    if (row_names):
        data.insert(0, '0', data.index, True)
    if (column_names):
        column_names_list = data.columns.to_list()
        data = data.to_numpy().tolist()
        all_data = [column_names_list] + data
    else:
        all_data = data.to_numpy().tolist()
    workbook = gc.open_by_key(googlesheet_ID)
    worksheet = workbook.worksheet(sheet_name)
    MAX_ROWS = 10
    for i in range(0, len(all_data), MAX_ROWS):
        chunk = all_data[i:i+MAX_ROWS]
        range_start = i + 1
        worksheet.update(range_name=f"A{range_start}", values=chunk)
    # worksheet.update(None,all_data)
    worksheet.clear()
    for i in range(0, len(all_data), MAX_ROWS):
        chunk = all_data[i:i+MAX_ROWS]
        range_start = i + 1
        worksheet.update(range_name=f"A{range_start}", values=chunk)
    # worksheet.update(None,all_data)
    return

#działa, opisana
def make_dataframe_backup(dataframe, backup_name, backup_path, row_names = True):
    now = datetime.now()
    dt_string = now.strftime("%Y-%m-%d_%Hh_%Mm_%Ss")
    backup_file_name = backup_name + "_" + dt_string
    gc.create(backup_file_name)
    data = dataframe.copy()
    if row_names:
        data.insert(0, '0', data.index, True)
    data = data.fillna('')

    all_data = data.to_numpy().tolist()
    worksheet = gc.open(backup_file_name).sheet1

    MAX_ROWS = 10
    for i in range(0, len(all_data), MAX_ROWS):
        chunk = all_data[i:i+MAX_ROWS]
        range_start = i + 1
        worksheet.update(range_name=f"A{range_start}", values=chunk)

    #worksheet.update(None,all_data)
    print(backup_path)
    sleep(30)

    default_save_path = "/content/drive/MyDrive/"
    !mv {default_save_path + backup_file_name + '.gsheet'} {backup_path}
    return

#działa, opisana
def make_archive_backup(archive_googlesheet_ID, backup_path):
    dataframe = get_googlesheet_by_name_dataframe(archive_googlesheet_ID)
    make_dataframe_backup(dataframe, 'archive', backup_path)
    return

#działa, opisana
#only to create, DO NOT USE AGAIN
def create_archive():
    data = get_all_sheets_from_googlesheet_dataframeList('1eM20dajVdOGcK_AZXDsj8ayzoPTjcRS65_LvrTfDPDI')
    data2 = merge_dataframes_list(data)
    data2 = data2.replace(',','.', regex=True)
    write_dataframe_to_googlesheet(data2, archive_googlesheet_ID, 'results')
    return

#działa, opisana
def add_results_to_archive_and_remove_duplicates(results_new, archive_googlesheet_ID):
    archive_old = get_googlesheet_by_name_dataframe(archive_googlesheet_ID)
    results_merged = merge_dataframes_list([archive_old, results_new])
    results_merged = results_merged.fillna('')
    archive_new = remove_dataframe_duplicates(results_merged, False)
    overwrite_googlesheet_by_dataframe(archive_new, archive_googlesheet_ID, 'results')
    return archive_new

#działa, opisana
def update_archive(results_dataframe, archive_googlesheet_ID, backup_path):
    make_archive_backup(archive_googlesheet_ID, backup_path)
    index_list = list(results_dataframe.index)
    parameters_index = [i for i, x in enumerate(index_list) if x == parameters_names[0]][0]
    df = results_dataframe.iloc[parameters_index:parameters_index+len(parameters_names)+1+11,-individuals_no:]
    df = add_results_to_archive_and_remove_duplicates(df, archive_googlesheet_ID)
    return df

#działa, opisana
def fill_dataframe_with_results_from_archive(results_dataframe, archive_dataframe):
    archive_numpy_array = archive_dataframe.to_numpy()
    results_numpy_array = results_dataframe.to_numpy()

    index_list = list(results_dataframe.index)
    parameters_index = index_list.index(parameters_names[0])
    parameters_no = len(parameters_names)
    v_i = index_list.index('L1 [cm/min]')

    for i in range(individuals_no):
        parameters = results_numpy_array[parameters_index:parameters_index+parameters_no,-1-i]
        archive_index = [j for j, param in enumerate(archive_numpy_array[0:8,:].T) if (param.astype(float) == parameters.astype(float)).all()]
        if len(archive_index) > 0:
            print('result detected: {}'.format(8-i))
            print(parameters)
            archive_data = archive_numpy_array[:,archive_index[0]].T[9:20]
            results_numpy_array[v_i:v_i+11,-1-i] = archive_data
    results_dataframe_new = pd.DataFrame(results_numpy_array, index = index_list)
    return results_dataframe_new

#działa, opisana ???
def save_new_iteration(results_dataframe, caterpillars_list_new, googlesheet_ID, sheet_name, archive_googlesheet_ID, backup_path):
    # results_dataframe_new = update_best_parameters(results_dataframe, **iteration_parameters)
    archive_dataframe = update_archive(results_dataframe, archive_googlesheet_ID, backup_path)

    results_dataframe_with_empty_column = add_empty_column_to_dataframe(results_dataframe)
    results_dataframe_with_new_interation = append_caterpillars_list_to_dataframe(results_dataframe_with_empty_column, caterpillars_list_new)

    results_dataframe_with_new_interation = fill_dataframe_with_results_from_archive(results_dataframe_with_new_interation, archive_dataframe)
    overwrite_googlesheet_by_dataframe(results_dataframe_with_new_interation, googlesheet_ID, sheet_name)
    return
# def save_new_iteration(results_dataframe, iteration_parameters, caterpillars_list_new, googlesheet_ID, sheet_name):
#     results_dataframe_new = update_best_parameters(results_dataframe, **iteration_parameters)
#     results_dataframe_with_empty_column = add_empty_column_to_dataframe(results_dataframe_new)
#     results_dataframe_with_new_interation = append_caterpillars_list_to_dataframe(results_dataframe_with_empty_column, caterpillars_list_new)

#     update_archive(results_dataframe, backup_path)
#     archive_dataframe = get_googlesheet_by_name_dataframe(archive_googlesheet_ID)
#     results_dataframe_with_new_interation = fill_dataframe_with_results_from_archive(results_dataframe_with_new_interation, archive_dataframe)
#     overwrite_googlesheet_by_dataframe(results_dataframe_with_new_interation, googlesheet_ID, sheet_name)
#     return

#działa, opisana
def PSO_step(PSO_params, individuals_no, googlesheet_ID, sheet_name, iteration_no, iteration_max, archive_googlesheet_ID, backup_path):
    results_dataframe = get_googlesheet_by_name_dataframe(googlesheet_ID, sheet_name)
    if iteration_no == 1:
        results_dataframe.index.values[0] = 'iteration (max = {})'.format(iteration_max)

    caterpillars_list_all = np.array([])
    for i in range(iteration_no-1):
        i_start = i * (individuals_no+1)
        i_stop = (1 + i) * (individuals_no+1) -1
        caterpillars_list_all = np.append(caterpillars_list_all, dataframe_to_caterpillars_list(results_dataframe[results_dataframe.columns[i_start:i_stop]]))
    # get new iteration
    caterpillars_list_new = get_new_iteration(PSO_params, individuals_no, caterpillars_list_all, iteration_no, iteration_max)

    save_new_iteration(results_dataframe, caterpillars_list_new, googlesheet_ID, sheet_name, archive_googlesheet_ID, backup_path)
    return

def new_generation():
    new_caterpillars_list = []
    random.seed()
    for i in range(0, individuals_no):
        cater = caterpillar()
        new_caterpillars_list.append(cater)
    return new_caterpillars_list

# PSO STEP

In [ ]:
# UPDATE ARCHIVE
googlesheet_ID = "1uRMavBXN2dD0QHs19RysfhgF-Jdtuh5gdG_Zcw3wboM" # PSO
sheet_name = "PSO_2"
archive_googlesheet_ID = '1R7loSptYksObERjJkEdwv_bL86h7F52IxaNqwZ2So9k'

backup_path = "/content/drive/MyDrive/PNaF/Diamentowy_Grant_2017-2021/Swimming_caterpillars_evolution/wyniki/backup/"
results_dataframe = get_googlesheet_by_name_dataframe(googlesheet_ID, sheet_name)

update_archive(results_dataframe, archive_googlesheet_ID, backup_path)



# PSO STEP

# PSO_params = [w, c1, c2]
PSO_params = [0, 0.2, 1.4]

iteration_no = 3
iteration_max = 5
backup_path = "/content/drive/MyDrive/PNaF/Diamentowy_Grant_2017-2021/Swimming_caterpillars_evolution/wyniki/backup/"

path = "/content/drive/MyDrive/PNaF/Diamentowy_Grant_2017-2021/Swimming_caterpillars_evolution/wyniki/"

sheet_name = "PSO_2"

PSO_step(PSO_params, individuals_no, googlesheet_ID, sheet_name, iteration_no, iteration_max, archive_googlesheet_ID, backup_path)

/content/drive/MyDrive/PNaF/Diamentowy_Grant_2017-2021/Swimming_caterpillars_evolution/wyniki/backup/


<ipython-input-9-de6016595c2a>:243: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  parameters = [column[name][0] for name in parameters_names]
<ipython-input-9-de6016595c2a>:250: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  parameters_best = [column[name][1] for name in parameters_names]


parameters: 0.4 12.0 1.0 50.0 1.0 3.0 1.0 82.5 
fitness: 1.802
velocities: -2.576185070662451 -6.182844169589883 0.44000000000000006 0.0 0.0 2.576185070662451 0.0 54.099886483911476 
best parameters: 0.4 12.0 1.0 50.0 1.0 3.0 1.0 82.5 
best fitness: 1.802 

r: 0.100900530504064
caterpillar 1 change duplicates, PSO step no: 1
r: -0.20612041658016134
caterpillar 3 change duplicates, PSO step no: 1
r: -0.7249395654739172
caterpillar 3 change duplicates, PSO step no: 2
r: -0.5423439822799621
caterpillar 4 change duplicates, PSO step no: 1
r: 0.5705400593187626
caterpillar 6 change duplicates, PSO step no: 1
caterpillar 7 change duplicates, PSO step no: 1
/content/drive/MyDrive/PNaF/Diamentowy_Grant_2017-2021/Swimming_caterpillars_evolution/wyniki/backup/
result detected: 5
[np.float64(0.3) np.float64(12.0) np.float64(1.0) np.float64(50.0)
 np.float64(1.0) np.float64(3.0) np.float64(1.0) np.float64(82.5)]
result detected: 4
[np.float64(0.1) np.float64(6.0) np.float64(0.2) np.float64(50.0)
 